In [1]:
import os
import pickle
import pandas as pd

# Directory containing the .pkl files
data_directory = '/Users/stevenwendel/Documents/GitHub/bg/data'

def pickle_to_dataframe(path):
    # Collect data in a list
    ga_raw = pickle.load(open(path, 'rb'))
    data = []
    
    for gen, runs in ga_raw.items():
        for run in runs:
            data.append([gen, run['dna'], run['dna_score']])
            
    # Create DataFrame in one go
    df = pd.DataFrame(data, columns=['generation', 'dna', 'dna_score'])
    return df

# List all .pkl files in the directory
all_data = []
for filename in os.listdir(data_directory):
    if filename.endswith('.pkl'):
        file_path = os.path.join(data_directory, filename)
        print(f'Opening {file_path}')
        df = pickle_to_dataframe(file_path)
        print(f'Closing {file_path}')
        all_data.append(df)
        print(f'Appended {file_path}')

# Concatenate all DataFrames into one
humongous_df = pd.concat(all_data, ignore_index=True)

# Sort the DataFrame by 'dna_score'
humongous_df = humongous_df.sort_values(by='dna_score', ascending=False, ignore_index=True)

# Display the combined DataFrame
display(humongous_df)

,generation,dna,dna_score
0,149,"[834, 0, 0, 1000, -7, -10, 0, 0, -20, 24, 0, 0...",984
1,146,"[708, 0, 0, 1000, -7, -10, 0, 0, -10, 14, 0, 0...",983
2,149,"[1000, 0, 0, 740, -54, -28, 40, 0, -24, 21, 0,...",983
3,148,"[563, 0, 0, 1000, -7, -912, 0, 0, -62, 15, 0, ...",983
4,148,"[520, 0, 0, 452, -32, -153, 10, 0, -46, 75, 0,...",983
...,...,...,...
18403085,119,"[26, 1, 188, 125, -259, -99, -309, -45, -158, ...",279
18403086,51,"[30, 2000, -10, 0, -44, -8, -46, 0, -300, -13,...",279
18403087,10,"[37, 2000, 2000, 2000, -443, -2000, -1163, -54...",278
18403088,103,"[157, 185, 58, 716, -29, -59, -108, -75, -477,...",260


In [ ]:
# Save the humongous_df to a new pkl
humongous_df[humongous_df['dna_score'] >= 720].to_pickle('humongous_df.pkl')


In [ ]:
import numpy as np

from src.neuron import *
from src.utils import *
from src.constants import * 
from src.network import *
from src.validation import *
from src.viz import *
from src.genetic_algorithm import *

# jh_weights= create_dna_string(new_jh_weights, ACTIVE_SYNAPSES)

# curr_dna = ga_results.iloc[581]['dna']
# curr_dna = jh_weights
# Load DNA
curr_sample = ga_results.iloc[581]
orig_dna = curr_sample['dna']
orig_score = curr_sample['dna_score']


new_dna = None
new_score = None

dna_matrix = load_dna(curr_dna)

"""
For each dna:
    

"""

# === Preparing Network === 
all_neurons = create_neurons()
splits, input_waves, alpha_array = create_experiment()
criteria_dict = define_criteria()
max_score = TMAX//BIN_SIZE * len(CRITERIA_NAMES)

dna_score, neuron_data = evaluate_dna(
                dna_matrix=dna_matrix,
                neurons=all_neurons,
                alpha_array=alpha_array,
                input_waves=input_waves,
                criteria=criteria_dict,
                curr_dna=curr_dna
                )
total_score = sum(dna_score.values())
print(f'    === DNA: {curr_dna}') 
print(f'    === Control: {dna_score["control"]}/{max_score}')
print(f'    === Experimental: {dna_score["experimental"]}/{max_score}')
print(f'    === Overall: {total_score}({total_score/(2*max_score):.2%})')
print('\n')

# STRUCTURE OF NEURON_DATA
# neuron_data = {
#     'experimental': {
#         'neuron_name': {
#             'hist_V': np.array(),
#             'spike_times': np.array()
#         },
#     'control': {
#         'neuron_name': {
#             'hist_V': np.array(),
#             'spike_times': np.array()
#             }
#         }
#     }
# }


# print(np.sum(neuron_data[ 'experimental']['VMresp']['spike_times'][3500:4000]))

# ====BINNING SPIKES===

# Define the bin size in terms of the number of time steps (assuming each step is 1ms)
bin_size = 1000

# Iterate over each condition and neuron to calculate spikes per bin
for condition in ['experimental', 'control']:
    print(f"Condition: {condition}")
    for neuron_name, data in neuron_data[condition].items():
        spike_times = data['spike_times']
        num_bins = len(spike_times) // bin_size
        spikes_per_bin = [int(np.sum(spike_times[i*bin_size:(i+1)*bin_size])) for i in range(num_bins)]
        
        print(f"Neuron: {neuron_name}, {spikes_per_bin}")

spikelist = neuron_data['experimental']['SNR1']['spike_times']
for i in range(len(spikelist)):
    if spikelist[i] == 1:
        print(i)



diagnostic = {
        'show_dna_matrix' : True,
        'show_neuron_plots' : True,
        'show_difference_histogram' : False,
        'show_dna_scores': False
    }


if diagnostic['show_dna_matrix']:
                print("Currently loaded matrix ---")
                display_matrix(dna_matrix, NEURON_NAMES)

if diagnostic['show_dna_scores']:
                print(f'{dna_score=}: {curr_dna}')
            
if diagnostic['show_neuron_plots']:
                for condition in ['experimental', 'control']:
                    target_neurons_hist_Vs = np.array([neuron_data[condition][name]['hist_V'] for name in NEURON_NAMES])
                    plot_neurons_interactive(hist_Vs=target_neurons_hist_Vs, neuron_names=NEURON_NAMES, sq_wave=input_waves[0], go_wave=input_waves[1], show_u=False)
                    